In [1]:
import glob
import pandas as pd
from Bio import SeqIO
from gzip import open as gopen
from collections import Counter

In [2]:
!mkdir -p cut_reads/16SrRNA

for f in glob.glob('raw_data/16SrRNA/AS*_S*_L001_R*_001.fastq.gz'):
    outf = 'cut_reads/16SrRNA/%s' % f.split('/')[-1]
    smpl = f.split('/')[-1].split('_')[0]
    records = SeqIO.parse(gopen(f,'rt'),'fastq')
    with gopen(outf,'wt') as hndl:
        for r in records:
            
            # Primer dimers were sequenced. They have monomorphic ends.
            # I skip sequences that were close to monomorphic in the last
            # 40 base pairs
            counts = Counter(str(r[110:150].seq).upper())
            proportions = {i: counts[i]/sum(counts.values()) for i in counts.keys()}
            if any([proportions[i] > 0.5 for i in proportions.keys()]):
                continue
                
            hndl.write(r[20:150].format('fastq'))

!mkdir -p merged/16SrRNA

df = pd.DataFrame(columns=['R1_count','R2_count',
                           'R1_mode_length','R1_count_at_mode_length',
                           'R2_mode_length','R2_count_at_mode_length',
                           'Mode_overlap','Count_at_mode_overlap'])

for f1 in glob.glob('cut_reads/16SrRNA/AS*_S*_L001_R1_001.fastq.gz'):
    f2 = f1.replace('_R1_','_R2_')
    
    smpl = f1.split('/')[-1].split('_')[0]
    
    merged = gopen('merged/16SrRNA/%s' % f1.split('/')[-1], 'wt')

    r1s = list(SeqIO.parse(gopen(f1,'rt'),'fastq'))
    r2s = SeqIO.to_dict(SeqIO.parse(gopen(f2,'rt'),'fastq'))
    
    count_r1 = len(r1s)
    count_r2 = len(r2s)
    
    overlapps = []
    r1_lengths= []
    r2_lengths= []
    
    for r1 in r1s:
        r1seq = str(r1.seq)
        r1_lengths.append(len(r1))
        try:
            r2seq = str(r2s[r1.id].seq.reverse_complement())
        except:
            continue
        r2_lengths.append(len(r2seq))
        over_len = 30
        r1_window = r1seq[-1*over_len:]
        r2_window = r2seq[:over_len]
        while over_len > 0 and r1_window != r2_window:
            over_len -= 1
            r1_window = r1seq[-1*over_len:]
            r2_window = r2seq[:over_len]
        overlapps.append(over_len)
        if over_len > 5:
            r2 = r2s[r1.id]
            m = r1 + r2.reverse_complement()[over_len:]
            m.id = r1.id
            m.description = r1.description
            merged.write(m.format('fastq'))
    merged.close()        
    if len(overlapps) == 0:
        print( smpl)
            
    ovlp_mode, ovlp_count_at_mode = sorted(Counter(overlapps).items(), key=lambda k: k[1], reverse=True)[0]
    r1len_mode, r1len_count_at_mode = sorted(Counter(r1_lengths).items(), key=lambda k: k[1], reverse=True)[0]
    r2len_mode, r2len_count_at_mode = sorted(Counter(r2_lengths).items(), key=lambda k: k[1], reverse=True)[0]
    df.loc[smpl] = [count_r1,count_r2,r1len_mode,r1len_count_at_mode,r2len_mode,r2len_count_at_mode,ovlp_mode,ovlp_count_at_mode]
df

,R1_count,R2_count,R1_mode_length,R1_count_at_mode_length,R2_mode_length,R2_count_at_mode_length,Mode_overlap,Count_at_mode_overlap
AS3006,2295,3073,130,2295,130,2189,8,1633
AS3041,9097,9895,130,9097,130,8827,8,6856
AS3019,11429,11774,130,11429,130,11129,8,8318
AS3000,19931,20262,130,19931,130,19463,8,15092
AS3014,14791,15110,130,14791,130,14488,8,11242
...,...,...,...,...,...,...,...,...
AS3071,9264,9490,130,9264,130,9055,8,7021
AS3073,4010,5033,130,4010,130,3865,8,3060
AS3005,14525,15240,130,14525,130,14115,8,10842
AS3063,11042,11668,130,11042,130,10705,8,8186


In [3]:
df.to_csv('merged_16SrRNA_read_stas.tsv', sep='\t')

In [5]:
!qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path merged/16SrRNA \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path merged/16SrRNA/demux.qza

In [9]:
!mkdir dada/16SrRNA
!qiime dada2 denoise-single \
  --i-demultiplexed-seqs merged/16SrRNA/demux.qza \
  --p-trim-left 0 \
  --p-trunc-len 250 \
  --p-max-ee 5.0 \
  --o-representative-sequences dada/16SrRNA/rep-seqs.qza \
  --o-table dada/16SrRNA/table.qza \
  --o-denoising-stats dada/16SrRNA/stats.qza

mkdir: cannot create directory ‘dada/16SrRNA’: File exists
Saved FeatureTable[Frequency] to: dada/16SrRNA/table.qza
Saved FeatureData[Sequence] to: dada/16SrRNA/rep-seqs.qza
Saved SampleData[DADA2Stats] to: dada/16SrRNA/stats.qza


In [10]:
!qiime feature-classifier classify-sklearn \
  --i-classifier DB/silva-132-99-515-806-nb-classifier.qza \
  --i-reads dada/16SrRNA/rep-seqs.qza \
  --o-classification dada/16SrRNA/taxonomy.qza

Saved FeatureData[Taxonomy] to: dada/16SrRNA/taxonomy.qza


### make tree

In [11]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences dada/16SrRNA/rep-seqs.qza \
  --o-alignment dada/16SrRNA/aligned-rep-seqs.qza \
  --o-masked-alignment dada/16SrRNA/masked-aligned-rep-seqs.qza \
  --o-tree dada/16SrRNA/unrooted-tree.qza \
  --o-rooted-tree dada/16SrRNA/rooted-tree.qza

Saved FeatureData[AlignedSequence] to: dada/16SrRNA/aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: dada/16SrRNA/masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: dada/16SrRNA/unrooted-tree.qza
Saved Phylogeny[Rooted] to: dada/16SrRNA/rooted-tree.qza
